# 基于动态并行性的快速排序算法

In [2]:
from __future__ import division
import numpy as np
from pycuda.compiler import DynamicSourceModule
import pycuda.autoinit
from pycuda import gpuarray
from random import shuffle

DynamicQuicksortCode='''
__device__ int partition(int * a, int lo, int hi)
{
 int i = lo;
 int pivot = a[hi];
 int temp;
 for (int k=lo; k<hi; k++)
 {
  if (a[k] < pivot)
  {
   temp = a[k];
   a[k] = a[i];
   a[i] = temp;
   i++;
  }
 }
 
 a[hi] = a[i];
 a[i] = pivot;
  
 return i;
}
  
__global__ void quicksort_ker(int *a, int lo, int hi)
{
 cudaStream_t s_left, s_right; 
 cudaStreamCreateWithFlags(&s_left, cudaStreamNonBlocking);
 cudaStreamCreateWithFlags(&s_right, cudaStreamNonBlocking);
 
 int mid = partition(a, lo, hi);
  
 if(mid - 1 - lo > 0)
   quicksort_ker<<< 1, 1, 0, s_left >>>(a, lo, mid - 1);
 if(hi - (mid + 1) > 0)
   quicksort_ker<<< 1, 1, 0, s_right >>>(a, mid + 1, hi);
    
 cudaStreamDestroy(s_left);
 cudaStreamDestroy(s_right);
}
'''

qsort_mod = DynamicSourceModule(DynamicQuicksortCode)

qsort_ker = qsort_mod.get_function('quicksort_ker')

if __name__ == '__main__':
    a = list(range(100))
    shuffle(a)
    
    a = np.int32(a)
    
    d_a = gpuarray.to_gpu(a)
    
    print('Unsorted array: %s' % a)
    
    qsort_ker(d_a, np.int32(0), np.int32(a.size - 1), grid=(1,1,1), block=(1,1,1))
    
    a_sorted = list(d_a.get())
    
    print('Sorted array: %s' % a_sorted)

Unsorted array: [ 9 33  7 30 71 17 10 41 42 70 82 44 18 45  3 46  6 19 89 34 27 23  5 14
 76 66 28 85 84 13 26  0 16 77 60 22 72 79 98 83 56 61 36 86 38 57 88 20
 50 53 74 97 92 37 64 67 12 58 69 32 35 24 87 52 11 47 90 96 29 31 51 40
 91 68 78 54  8 59 39 94 63 75 95  2 25 55 80  1 49 99 21 81 93 73 43 65
 15 48  4 62]
Sorted array: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
